# Project Group - 15

Members: 
Sören Burghardt, 
Sarah Blanc, 
Allan Guzmann, 
Lars van den Berg, 
Simon Schreders

Student numbers: 
5861012, 
5854830, 
5718619,
5626668,
5878845

# Research Objective

What are the effects on aviation that COVID-19  caused at the Amsterdam Schiphol Airport (EHAM) and its most frequented destinations for freight and passenger transport?

When COVID-19 became a global pandemic in 2020, the containments plunged the global economy into an unprecedented recession. The magnitude of the economic shock was such that it resulted in unrecognisable data and a "data fog" that made it difficult to interpret and, more importantly, to predict. The last two years have been extraordinary, both for global economy and the humanity. While the COVID-19 pandemic now appears to be under control thanks to vaccination programmes, parts of the global economy are not yet fully recovered. As a result of the protective travel restrictions, the aviation sector has been hit harder than many others industries. In this project, our aim was to observe the impacts that covid has had on aviation and in particular, the impacts on Amsterdam Schiphol Airport and its most frequented destinations. This from the perspective of passenger and freight transport.

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

Sören: coding

Sarah: coding

Allan: coding

Lars: coding

Simon: coding

# Data Used

COVID-19 cases in the world (Our World in Data)

Flight data passengers (Eu Stat)

Flight data freight (EU Stat)

OD Data for Airport Amsterdam (EU Stat)


# Data Pipeline

All the data for flights from Amsterdam Airport Schiphol (EHAM) were grouped by country. This way two top 5 were created for both the volume of cargo and mail and the passangers. The countries from these two top 5's and the Netherlands were used to filter the COVID-19 cases, using the amount of confirmed new cases per million in a country. 

As a first step, we are going to import all the librairies that we will need.

In [2]:
import pandas as pd
import numpy as np
import plotly as plt

import math
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

# Part I - Data import

First, we are going to import the dataframes. We are going to modifiy them in order to prepare them for our project and to combine them together. 

Let's start off with the covid  data.

In [3]:
#COVID PART 
filepath = 'Data_Sets/owid-covid-data.csv'
covid_data = pd.read_csv(filepath, delimiter=',')

# Select the columns of interest from the original dataset and filter it, also expand the date to accomodate year and month
columns_of_interest = ['new_cases_smoothed_per_million','people_fully_vaccinated_per_hundred','date','location']
filtered_covid_data = covid_data[columns_of_interest]
filtered_covid_data[['Year','Month','Day']] = filtered_covid_data.date.str.split("-",expand=True)

# Selecting the countries of interest for cargo and passenger trasnport
cargo_countries_of_interest =['China','United States','United Arab Emirates','Brazil','Qatar','Netherlands']
passenger_countries_of_interest = ['United Kingdom','Spain','Germany','Italy','United States','Netherlands']

# Group by year, month and location to calculate the mean for cargo transport
cargo_filtered_covid_data = filtered_covid_data[filtered_covid_data['location'].isin(cargo_countries_of_interest)]
cargo_filtered_covid_data = cargo_filtered_covid_data.groupby(['location','Year','Month']).mean()

#Resetting the index and creating new collum with Year and Month 
cargo_filtered_covid_data = cargo_filtered_covid_data.reset_index()
cargo_filtered_covid_data['date'] = cargo_filtered_covid_data['Year'] + ['-'] + cargo_filtered_covid_data['Month'] 

# Group by year, month and location to calculate the mean for cargo transport
passenger_filtered_covid_data = filtered_covid_data[filtered_covid_data['location'].isin(passenger_countries_of_interest)]
passenger_filtered_covid_data = passenger_filtered_covid_data.groupby(['location','Year','Month']).mean()

#Resetting the index and creating new collum with Year and Month 
passenger_filtered_covid_data = passenger_filtered_covid_data.reset_index()
passenger_filtered_covid_data['date'] = passenger_filtered_covid_data['Year'] + ['-'] + passenger_filtered_covid_data['Month']
passenger_filtered_covid_data.head()



c:\Users\soren\anaconda3\envs\TIL6022v\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,location,Year,Month,new_cases_smoothed_per_million,people_fully_vaccinated_per_hundred,date
0,Germany,2020,01,NaN,NaN,2020-01
1,Germany,2020,02,0.011621,NaN,2020-02
2,Germany,2020,03,18.258710,NaN,2020-03
3,Germany,2020,04,43.080800,NaN,2020-04
4,Germany,2020,05,9.641290,NaN,2020-05


After implementing the covid data, we have modified them according to our needs. First of all we decided to keep only the columns we needed to avoid having unused information in this data frame. We filtered this new data frame and expanded the date to the accomodate year and month.

Then the aviation part.

In [2]:
freight = pd.read_csv("Data_Sets/avia_gor_nl__custom_3564729_monthly_linear.csv")
passengers = pd.read_csv("Data_Sets/avia_par_nl__custom_3564728_monthly_linear.csv")
country_code_fix = {'EL':'GR','AN':'BQ'}

def top_n_year(df,n,year):
    df[["airp_country_1","airp_code_1","airp_country_2","airp_code_2"]] = df.airp_pr.str.split("_",expand=True)
    df[["Year","Month"]] = df.TIME_PERIOD.str.split("-",expand=True)
    topn = df.groupby(['Year',"airp_country_2"]).sum()
    topn = topn.loc[year,:].sort_values("OBS_VALUE", ascending= False).head(n).reset_index()
    cleaned = df[df["airp_country_2"].isin(topn["airp_country_2"])]
    return cleaned

def codes_correction(df,ISO_2):
    df[ISO_2] = df[ISO_2].replace(to_replace= country_code_fix)
    df['iso_3_country'] = coco.convert(names = df[ISO_2], to= 'ISO3')
    df['country_2_name'] = coco.convert(names = df[ISO_2], to= 'name_short')
    return df

top_passengers = top_n_year(passengers,5,'2019')
top_cargo = top_n_year(freight,5,'2019')

SyntaxError: invalid syntax (3580322668.py, line 1)

In [ ]:
total_cargo = top_cargo.groupby(['airp_country_2','Year']).sum()
total_cargo = total_cargo.reset_index()
total_cargo = codes_correction(total_cargo,'airp_country_2')
fig = px.line(total_cargo, x='Year', y= 'OBS_VALUE' ,color='airp_country_2')
fig.show()

In [ ]:
total_passengers = top_passengers.groupby(['airp_country_2','Year']).sum()
total_passengers = total_passengers.reset_index()
total_passengers = codes_correction(total_passengers,'airp_country_2')
fig = px.line(total_passengers, x='Year', y= 'OBS_VALUE' ,color='airp_country_2')
fig.show()

In [ ]:
map_data_passengers = world.merge(passengers_for_map,left_on = 'iso_a3', right_on = 'iso_3_country', how = 'left')

fig = px.choropleth(map_data_passengers, hover_name = 'country_2_name',locations= 'iso_3_country', locationmode= 'ISO-3', color='OBS_VALUE')

fig.update_layout(
    title_text = 'Total passengers flying from Schiphol Airport in 2019',
    geo= dict(
        showframe = False,
        showcoastlines = False,
        projection_type = 'equirectangular'
    )
)
fig.show()

In [ ]:
map_data_cargo = world.merge(freight_for_map,left_on = 'iso_a3', right_on = 'iso_3_country', how = 'left')

fig = px.choropleth(freight_for_map, hover_name = 'country_2_name',locations= 'iso_3_country', locationmode= 'ISO-3', color='OBS_VALUE')

fig.update_layout(
    title_text = 'Total tons transported by air from Schiphol Airport in 2019',
    geo= dict(
        showframe = False,
        showcoastlines = False,
        projection_type = 'equirectangular'
    )
)
fig.show()

In [ ]:
fig = px.bar(map_data_passengers.sort_values('OBS_VALUE',ascending= False).head(20),x= 'country_2_name', y = 'OBS_VALUE')
fig.show()

In [ ]:
fig = px.bar(map_data_cargo.sort_values('OBS_VALUE',ascending= False).head(20),x= 'country_2_name', y = 'OBS_VALUE')
fig.show()

After implementing the aviation data, we have modified them according to our needs again. we have kept only the rows containing information on the top 5 countries (i.e. the countries that are studied) for the passenger and cargo transports.

Then, the merging was quite easy:

In [ ]:
MERGING PART

# Part II - Data Processing

In this second part, we are going to extract the most interesting and the main information out ouf our data. We are not going to do any comparisons between countries or airports (comparisons that we are going to do in task 3).

First, let's show a map containing the five routes that carried the most passengers from Amsterdam airport in 2019.

In [ ]:
CODE

First, let's show a map containing the five routes that carried the most freigh amount from Amsterdam airport in 2019.

In [ ]:
(0)

As we can see, while passenger transport is dominated by intra-European transport, cargo transport is more international

(1)

In [ ]:
(1)

Let's now take a look at the evolution of the number of passengers and the amount of cargo transported from Schipol to each of the top 5 destinations. The time period used for these graphs is from 2000 to now.

In [ ]:
(2)

In parallel, let's look at the evolution of covid cases since the beginning of the pandemic and the vaccination rate in the Netherlands.

In [6]:
fig = px.line(passenger_filtered_covid_data, x='date', y= 'new_cases_smoothed_per_million' ,color='location',
                title= 'New covid cases in the TOP 5 countrys for passenger transport')

fig.show()

In [8]:
fig = px.line(cargo_filtered_covid_data, x='date', y= 'new_cases_smoothed_per_million' ,color='location',
              title= 'New covid cases in the TOP 5 countrys for cargo transport', range_y=[0,2000] )

fig.show()

In [9]:
fig = px.line(passenger_filtered_covid_data, x='date', y= 'people_fully_vaccinated_per_hundred' ,color='location',
               title= 'Vaccination rate in percent in the TOP 5 countrys for passenger transport' )

fig.show()

In [10]:
fig = px.line(cargo_filtered_covid_data, x='date', y= 'people_fully_vaccinated_per_hundred' ,color='location',
                title= 'Vaccination rate in percent in the TOP 5 countrys for cargo transport')

fig.show()

It is interesting to relate the number of Covid cases in the Netherlands to all passenger departures from Amsterdam airport. Similarly, the link between the vaccination rate and passenger departures from Amsterdam airport may provide useful information.

In [ ]:
(4) (5)

peaks covid cases NL vs number of passenger leaving NL (see previous work to see hom to find common peaks)

valleys covid cases NL vs number of passengers leaving  NL (see previous work to see hom to find common peaks)

common peaks and valleys between NL and ... for number of passengers and number of freigh (see previous work to see hom to find common peaks)
goal here: to see if covid had the same impact (peaks) at the same time in 2 or more countries

# Part 3 - Data Visualisation

In this last part, we are going to do a little research and a little comparison about China.

In [ ]:
CODE